In [228]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [229]:
disasters = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/disasters_2016.csv')
codes = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/country_codes.csv')
antirating_1 = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/antirating_v0.csv')
donations = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/data_merged_2006-2022.csv')
conflicts = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/conflicts_2016.csv') ## мб плохой источник
gini = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/gini_worldbank.csv',skiprows=3)

# conflicts_ = pd.read_csv(r"C:\Users\filim\Downloads\time_series_short_new_0.xls")

In [3]:
antirating_1

,Year,Code,Access to electricity (% of population)_x,Access to electricity (% of population)_y,Primary energy consumption per capita (kWh/person),Life expectancy,People using at least basic sanitation services (% of population),Annual CO2 emissions (per capita),"Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))",GDP per capita,417485-annotations,Population (historical estimates),"Child mortality (Select Gapminder, v10) (2017)",Prevalence of undernourishment (% of population),People using at least basic drinking water services (% of population)
0,2016,ALB,29.0,29.0,20.0,46.0,50.0,11.0,30.0,16.0,74.0,12.0,24.0,27.0,58.0
1,2016,ARM,27.0,27.0,21.0,20.0,32.0,15.0,43.0,15.0,74.0,11.0,25.0,32.0,28.0
2,2016,AUS,74.0,74.0,67.0,71.0,74.0,74.0,68.0,69.0,74.0,50.0,58.0,74.0,17.0
3,2016,AUT,74.0,74.0,59.0,56.0,73.0,55.0,52.0,62.0,74.0,32.0,62.0,74.0,14.0
4,2016,BGD,5.0,5.0,4.0,12.0,4.0,4.0,4.0,4.0,74.0,69.0,8.0,6.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2016,UKR,74.0,74.0,36.0,11.0,41.0,42.0,40.0,14.0,74.0,55.0,34.0,74.0,60.0
70,2016,GBR,74.0,74.0,48.0,54.0,54.0,45.0,68.0,60.0,74.0,61.0,49.0,74.0,14.0
71,2016,USA,74.0,74.0,71.0,48.0,66.0,73.0,73.0,70.0,74.0,72.0,40.0,74.0,29.0
72,2016,URY,25.0,25.0,28.0,42.0,35.0,17.0,21.0,38.0,74.0,15.0,33.0,74.0,34.0


## Катастрофы

In [230]:
disasters_ = disasters[["Dis No", "Country", "ISO", "Disaster Group", "Disaster Subgroup", "Disaster Type", "Total Damages ('000 US$)", "Total Deaths"]]
disasters_

,Dis No,Country,ISO,Disaster Group,Disaster Subgroup,Disaster Type,Total Damages ('000 US$),Total Deaths
0,2016-0175-BGD,Bangladesh,BGD,Natural,Meteorological,Storm,600000.0,28.0
1,2016-0117-ECU,Ecuador,ECU,Natural,Geophysical,Earthquake,2000000.0,672.0
2,2016-0095-CHN,China,CHN,Natural,Hydrological,Flood,170000.0,5.0
3,2016-0137-CHN,China,CHN,Natural,Hydrological,Flood,97000.0,20.0
4,2016-0041-FJI,Fiji,FJI,Natural,Meteorological,Storm,600000.0,45.0
...,...,...,...,...,...,...,...,...
528,2016-0446-SRB,Serbia,SRB,Natural,Hydrological,Flood,NaN,NaN
529,2016-0575-SSD,South Sudan,SSD,Natural,Biological,Epidemic,NaN,15.0
530,2016-0327-SSD,South Sudan,SSD,Natural,Hydrological,Flood,NaN,NaN
531,2016-0578-BEN,Benin,BEN,Technological,Technological,Miscellaneous accident,NaN,26.0


In [231]:
# оставим в датасете наших критериев благосостояния только те страны, которые участвуют в CBPF
res1 = pd.merge(donations, codes, how="left", left_on="Donor Name", right_on="Entity")
res1 = res1.drop(columns=["Entity"], axis=1)
res2 = pd.merge(res1, codes, how="left", left_on="CBPF Name", right_on="Entity")

countries = list(res2.Code_x.unique())
countries.extend(list(res2.Code_y.unique()))

In [232]:
disasters_ = disasters_[disasters_["ISO"].isin(countries)]
disasters_.head()

,Dis No,Country,ISO,Disaster Group,Disaster Subgroup,Disaster Type,Total Damages ('000 US$),Total Deaths
0,2016-0175-BGD,Bangladesh,BGD,Natural,Meteorological,Storm,600000.0,28.0
5,2016-0112-AFG,Afghanistan,AFG,Natural,Hydrological,Flood,NaN,30.0
6,2016-0191-AFG,Afghanistan,AFG,Natural,Hydrological,Flood,NaN,6.0
8,2016-0077-BRA,Brazil,BRA,Natural,Hydrological,Flood,100000.0,30.0
11,2016-0101-ETH,Ethiopia,ETH,Natural,Hydrological,Flood,NaN,28.0


In [233]:
disasters_.groupby(['ISO', 'Disaster Group']).count()["Country"].reset_index()

,ISO,Disaster Group,Country
0,AFG,Natural,4
1,AFG,Technological,2
2,ARM,Natural,1
3,AUS,Natural,4
4,BDI,Natural,1
...,...,...,...
75,USA,Technological,6
76,VEN,Natural,1
77,VEN,Technological,1
78,YEM,Natural,3


In [236]:
dis = pd.DataFrame(disasters_.groupby(['ISO']).count()[['Disaster Group', 'Total Deaths']]).reset_index()
dis.rename(columns={'Disaster Group': 'qty_disasters_2016', 'Total Deaths_2016': 'qty_deaths', 'ISO': 'Code'}, inplace=True)
disaster_ranked = dis.set_index('Code').rank(ascending=False)
print(dis.shape)
dis.head()

(51, 3)


,Code,qty_disasters_2016,Total Deaths
0,AFG,6,6
1,ARM,1,0
2,AUS,4,3
3,BDI,3,3
4,BEL,2,2


## Конфликты

In [237]:
conflicts.columns

Index(['conflict_new_id', 'conflict_name', 'dyad_new_id', 'dyad_name',
       'type_of_violence', 'active_year', 'country', 'conflict_dyad',
       'dyad_ep_start', 'dyad_ep_end', 'dyad_recurrence_date', 'dyad_ep_id',
       'conf_ep_start', 'conf_ep_end', 'conf_recurrence_date', 'conf_ep_id',
       'location', 'incompatibility', 'territory_name', 'type_of_conflict',
       'region', 'conf_subid', 'conf_recurrence', 'confactor_recurrence',
       'dyad_recurrence', 'factions', 'link_type_1', 'link_id_1',
       'link_type_2', 'link_id_2', 'link_type_3', 'link_id_3', 'link_type_4',
       'link_id_4', 'link_type_5', 'link_id_5', 'link_type_6', 'link_id_6',
       'dyad_recurrence_years', 'conf_recurrence_years', 'conf_ep_low',
       'conf_ep_best', 'conf_ep_high', 'dyad_ep_low', 'dyad_ep_best',
       'dyad_ep_high', 'conf_ep_freq', 'dyad_ep_freq', 'cont_dyad',
       'cont_conf'],
      dtype='object')

In [238]:
if  conflicts.sort_values('conf_ep_start').any().any():
    print('есть непонятные nan!!')
# conflicts.sort_values('conf_ep_start')[-11:]

есть непонятные nan!!


In [262]:
conflicts_ = conflicts[['country', 'conf_ep_start', 'conf_ep_end', 'conf_ep_freq']]
conflicts_['conf_ep_end'] = conflicts_['conf_ep_end'].fillna('11/8/2022')
conflicts_['conf_ep_start'] = pd.to_datetime(conflicts_['conf_ep_start'])
conflicts_['conf_ep_end'] = pd.to_datetime(conflicts_['conf_ep_end'])
is_2016 = (conflicts_['conf_ep_start'] <= pd.datetime(2016,1,1)) & (pd.datetime(2016,1,1) <= conflicts_['conf_ep_end'])
conflicts_['conf_ep_end'] = conflicts_['conf_ep_end']

conflicts_ = conflicts_[is_2016].dropna() #!!!!!!!! дропаю все nan без разбора

conflicts_['qty_conf_2016'] = 1
conflicts_ = conflicts_.rename(columns={'conf_ep_freq':'freq_conf_2016'})
if conflicts_.isna().any().any():
    print('__________KAMRAN: Есть пустые!________')
    display(conflicts_.sort_values('conf_ep_start'))
    raise Exception('__________KAMRAN: Есть пустые!________') 

conflicts_ = conflicts_[is_2016].groupby('country').agg({'freq_conf_2016':'mean','qty_conf_2016':'sum'}).reset_index()
conflicts_ = conflicts_[conflicts_['country']!='Somalia'] #!!!!!!!! дропаю Сомали

country_rename = {'Myanmar (Burma)': 'Myanmar', 
           'Russia (Soviet Union)': 
           'Russia','Yemen (North Yemen)':'Yemen',
           'South Sudan':'Former Sudan',
           'Sudan':'Former Sudan',
           'DR Congo (Zaire)':'Democratic Republic of Congo'}

conflicts_['country'] = conflicts_['country'].replace(country_rename)

if (set(conflicts_.country) - set(codes.Entity)):
    print('_________Есть страны, которые не заджойнятся справа_________')
    display(set(conflicts_.country) - set(codes.Entity))
    raise Exception('_________Есть страны, которые не заджойнятся справа_________') 
    
conflicts_ = pd.merge(conflicts_, codes, how="left", left_on="country", right_on="Entity").drop(columns=['Unnamed: 0','country','Entity'])

conflicts_ranked = conflicts_.set_index('Code').rank(ascending=False).head()
print(conflicts_.shape)
conflicts_.head()


(37, 3)


,freq_conf_2016,qty_conf_2016,Code
0,23000.764706,17,AFG
1,3253.142857,7,DZA
2,48.000000,2,AGO
3,276.000000,1,AZE
4,31.000000,1,BGD


## gini

In [306]:
(gini.mean(axis=1)+gini['2016'].fillna(0))[:252]

0            NaN
1            NaN
2            NaN
3            NaN
4      48.666667
         ...    
247    85.460000
248    52.604348
249          NaN
250    81.633333
251    81.159375
Length: 252, dtype: float64

In [294]:
gini[['Country Code','2016']].fillna(0)[:252]

,Country Code,2016
0,ABW,0.0
1,AFE,0.0
2,AFG,0.0
3,AFW,0.0
4,AGO,0.0
...,...,...
247,UGA,42.8
248,UKR,25.0
249,UMC,0.0
250,URY,39.7


In [311]:
gini_  = gini.copy()
isna_2016 = gini_['2016'].isna()

gini_['gini_2016'] = gini_['2016'].fillna(gini_[isna_2016].mean(axis=1)) #!!!!!! криво заменяю NaN на среднее где нет инфы по 2016
gini_ = gini_.dropna(subset='gini_2016') #дропаю NaN

gini_ = gini_[['gini_2016','Country Code']]
display(gini[gini['Country Code']=='USA'][isna_2016].mean(axis=1))
gini_ = gini_.rename(columns={'Country Code':'Code'})

gini_ranked = gini_.set_index('Code').sort_values('gini_2016').rank(ascending=False)
print(gini_.shape)
gini_.head()

Series([], dtype: float64)

(168, 2)


,gini_2016,Code
4,48.666667,AGO
5,33.700000,ALB
8,29.250000,ARE
9,42.000000,ARG
10,32.500000,ARM


## Merging

Объеденяем следующие 4ре таблицы по стране

In [312]:
display(antirating_1.head(), disaster_ranked.head(), conflicts_ranked.head(), gini_ranked.head())

,Year,Code,Access to electricity (% of population)_x,Access to electricity (% of population)_y,Primary energy consumption per capita (kWh/person),Life expectancy,People using at least basic sanitation services (% of population),Annual CO2 emissions (per capita),"Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))",GDP per capita,417485-annotations,Population (historical estimates),"Child mortality (Select Gapminder, v10) (2017)",Prevalence of undernourishment (% of population),People using at least basic drinking water services (% of population)
0,2016,ALB,29.0,29.0,20.0,46.0,50.0,11.0,30.0,16.0,74.0,12.0,24.0,27.0,58.0
1,2016,ARM,27.0,27.0,21.0,20.0,32.0,15.0,43.0,15.0,74.0,11.0,25.0,32.0,28.0
2,2016,AUS,74.0,74.0,67.0,71.0,74.0,74.0,68.0,69.0,74.0,50.0,58.0,74.0,17.0
3,2016,AUT,74.0,74.0,59.0,56.0,73.0,55.0,52.0,62.0,74.0,32.0,62.0,74.0,14.0
4,2016,BGD,5.0,5.0,4.0,12.0,4.0,4.0,4.0,4.0,74.0,69.0,8.0,6.0,44.0


,qty_disasters_2016,Total Deaths
Code,,
AFG,14.5,13.5
ARM,45.5,48.5
AUS,22.0,25.0
BDI,27.0,25.0
BEL,34.5,31.5


,freq_conf_2016,qty_conf_2016
Code,,
AFG,1.0,4.0
DZA,4.0,12.0
AGO,31.0,24.5
AZE,23.0,32.5
BGD,32.0,32.5


,gini_2016
Code,
SVN,168.0
UKR,167.0
SVK,166.0
BLR,165.0
CZE,164.0


In [325]:
antirating_2 = pd.merge(antirating_1, disaster_ranked.reset_index(), how="left", left_on="Code", right_on="Code")
antirating_2 = pd.merge(antirating_2, conflicts_ranked.reset_index(), how="left", left_on="Code", right_on="Code")
antirating_2 = pd.merge(antirating_2, gini_ranked.reset_index(), how="left", left_on="Code", right_on="Code")
if antirating_2.gini_2016.isna().any():
    print('__________KAMRAN: Gini соединилось с NaN!________')
    display(antirating_2.sort_values('gini_2016'))
    raise Exception('__________KAMRAN: Gini соединилось с NaN!________') 
antirating_2 = antirating_2.fillna(len(antirating_2)) ## заменяю всё на максимальный антирейтинг, так так нет конфликта/катастрофы <=> 0 катастроф/конфликтов  <=> антирейтинг максимальный

In [326]:
print(antirating_2.shape)
antirating_2.head()

(74, 20)


,Year,Code,Access to electricity (% of population)_x,Access to electricity (% of population)_y,Primary energy consumption per capita (kWh/person),Life expectancy,People using at least basic sanitation services (% of population),Annual CO2 emissions (per capita),"Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))",GDP per capita,417485-annotations,Population (historical estimates),"Child mortality (Select Gapminder, v10) (2017)",Prevalence of undernourishment (% of population),People using at least basic drinking water services (% of population),qty_disasters_2016,Total Deaths,freq_conf_2016,qty_conf_2016,gini_2016
0,2016,ALB,29.0,29.0,20.0,46.0,50.0,11.0,30.0,16.0,74.0,12.0,24.0,27.0,58.0,74.0,74.0,74.0,74.0,122.0
1,2016,ARM,27.0,27.0,21.0,20.0,32.0,15.0,43.0,15.0,74.0,11.0,25.0,32.0,28.0,45.5,48.5,74.0,74.0,130.0
2,2016,AUS,74.0,74.0,67.0,71.0,74.0,74.0,68.0,69.0,74.0,50.0,58.0,74.0,17.0,22.0,25.0,74.0,74.0,122.0
3,2016,AUT,74.0,74.0,59.0,56.0,73.0,55.0,52.0,62.0,74.0,32.0,62.0,74.0,14.0,74.0,74.0,74.0,74.0,145.5
4,2016,BGD,5.0,5.0,4.0,12.0,4.0,4.0,4.0,4.0,74.0,69.0,8.0,6.0,44.0,12.0,10.5,32.0,32.5,131.5


In [ ]:
antirating_2.to_csv('Data/antirating_v2.csv')